# 如何序列化提示

通常将提示存储为文件而不是Python代码会更好。这样易于共享、存储和版本化提示。本笔记本介绍了如何在LangChain中执行此操作，演示了所有不同类型的提示和不同的序列化选项。

在高层次上，序列化应用以下设计原则：

1. 支持JSON和YAML。我们希望支持在磁盘上人类可读的序列化方法，而YAML和JSON是最流行的两种方法。请注意，此规则适用于提示。对于其他资源，例如示例，可能支持不同的序列化方法。

2. 我们支持在一个文件中指定所有内容，或将不同组件（模板、示例等）存储在不同的文件中并引用它们。对于某些情况，将所有内容存储在单个文件中可能是最合适的，但对于其他情况，将某些资源拆分（长模板、大型示例、可重用组件）可能更好。LangChain支持两种方式。

还有一种从磁盘加载提示的单个入口点，可以轻松加载任何类型的提示。

In [1]:
# All prompts are loaded through the `load_prompt` function.
from langchain.prompts import load_prompt

## PromptTemplate

本节介绍了一些加载PromptTemplate的示例。

### 从YAML加载
这显示了从YAML加载PromptTemplate的示例。

In [2]:
!cat simple_prompt.yaml

_type: prompt
input_variables:
    ["adjective", "content"]
template: 
    Tell me a {adjective} joke about {content}.


In [3]:
prompt = load_prompt("simple_prompt.yaml")
print(prompt.format(adjective="funny", content="chickens"))

Tell me a funny joke about chickens.


### 从JSON加载
这显示了从JSON加载PromptTemplate的示例。

In [4]:
!cat simple_prompt.json

{
    "_type": "prompt",
    "input_variables": ["adjective", "content"],
    "template": "Tell me a {adjective} joke about {content}."
}


In [ ]:
prompt = load_prompt("simple_prompt.json")
print(prompt.format(adjective="funny", content="chickens"))

告诉我一个有关鸡的有趣笑话。

### 从文件中加载模板
这个例子展示了将模板存储在单独的文件中，然后在配置中引用它的示例。请注意，键从template更改为template_path。

In [5]:
!cat simple_template.txt

Tell me a {adjective} joke about {content}.

In [6]:
!cat simple_prompt_with_template_file.json

{
    "_type": "prompt",
    "input_variables": ["adjective", "content"],
    "template_path": "simple_template.txt"
}


In [7]:
prompt = load_prompt("simple_prompt_with_template_file.json")
print(prompt.format(adjective="funny", content="chickens"))

Tell me a funny joke about chickens.


## FewShotPromptTemplate

本节展示了一些加载FewShotPromptTemplate的示例。

### 示例
这显示了存储为json的示例的样子。

In [8]:
!cat examples.json

[
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"}
]


这是将相同示例存储为yaml的示例。

In [9]:
!cat examples.yaml

- input: happy
  output: sad
- input: tall
  output: short


### 从YAML加载
这显示了从YAML加载FewShot示例的示例。

In [10]:
!cat few_shot_prompt.yaml

_type: few_shot
input_variables:
    ["adjective"]
prefix: 
    Write antonyms for the following words.
example_prompt:
    _type: prompt
    input_variables:
        ["input", "output"]
    template:
        "Input: {input}\nOutput: {output}"
examples:
    examples.json
suffix:
    "Input: {adjective}\nOutput:"


In [11]:
prompt = load_prompt("few_shot_prompt.yaml")
print(prompt.format(adjective="funny"))

Write antonyms for the following words.

Input: happy
Output: sad

Input: tall
Output: short

Input: funny
Output:


如果您从yaml文件中加载示例，则也会起作用。

In [12]:
!cat few_shot_prompt_yaml_examples.yaml

_type: few_shot
input_variables:
    ["adjective"]
prefix: 
    Write antonyms for the following words.
example_prompt:
    _type: prompt
    input_variables:
        ["input", "output"]
    template:
        "Input: {input}\nOutput: {output}"
examples:
    examples.yaml
suffix:
    "Input: {adjective}\nOutput:"


In [13]:
prompt = load_prompt("few_shot_prompt_yaml_examples.yaml")
print(prompt.format(adjective="funny"))

Write antonyms for the following words.

Input: happy
Output: sad

Input: tall
Output: short

Input: funny
Output:


### 从JSON加载
这显示了从JSON加载FewShot示例的示例。

In [14]:
!cat few_shot_prompt.json

{
    "_type": "few_shot",
    "input_variables": ["adjective"],
    "prefix": "Write antonyms for the following words.",
    "example_prompt": {
        "_type": "prompt",
        "input_variables": ["input", "output"],
        "template": "Input: {input}\nOutput: {output}"
    },
    "examples": "examples.json",
    "suffix": "Input: {adjective}\nOutput:"
}   


In [15]:
prompt = load_prompt("few_shot_prompt.json")
print(prompt.format(adjective="funny"))

Write antonyms for the following words.

Input: happy
Output: sad

Input: tall
Output: short

Input: funny
Output:


### 在配置中的示例
这显示了直接在配置中引用示例的示例。

In [16]:
!cat few_shot_prompt_examples_in.json

{
    "_type": "few_shot",
    "input_variables": ["adjective"],
    "prefix": "Write antonyms for the following words.",
    "example_prompt": {
        "_type": "prompt",
        "input_variables": ["input", "output"],
        "template": "Input: {input}\nOutput: {output}"
    },
    "examples": [
        {"input": "happy", "output": "sad"},
        {"input": "tall", "output": "short"}
    ],
    "suffix": "Input: {adjective}\nOutput:"
}   


In [17]:
prompt = load_prompt("few_shot_prompt_examples_in.json")
print(prompt.format(adjective="funny"))

Write antonyms for the following words.

Input: happy
Output: sad

Input: tall
Output: short

Input: funny
Output:


### 从文件中加载示例提示
这个例子展示了从单独的文件中加载用于格式化示例的PromptTemplate的示例。请注意，键从example_prompt更改为example_prompt_path。

In [1]:
!cat example_prompt.json

{
    "_type": "prompt",
    "input_variables": ["input", "output"],
    "template": "Input: {input}\nOutput: {output}" 
}


In [19]:
!cat few_shot_prompt_example_prompt.json 

{
    "_type": "few_shot",
    "input_variables": ["adjective"],
    "prefix": "Write antonyms for the following words.",
    "example_prompt_path": "example_prompt.json",
    "examples": "examples.json",
    "suffix": "Input: {adjective}\nOutput:"
}   


In [20]:
prompt = load_prompt("few_shot_prompt_example_prompt.json")
print(prompt.format(adjective="funny"))

Write antonyms for the following words.

Input: happy
Output: sad

Input: tall
Output: short

Input: funny
Output:


## PromptTempalte with OutputParser
这显示了从文件加载带有OutputParser的提示的示例。

In [13]:
! cat prompt_with_output_parser.json

{
    "input_variables": [
        "question",
        "student_answer"
    ],
    "output_parser": {
        "regex": "(.*?)\\nScore: (.*)",
        "output_keys": [
            "answer",
            "score"
        ],
        "default_output_key": null,
        "_type": "regex_parser"
    },
    "partial_variables": {},
    "template": "Given the following question and student answer, provide a correct answer and score the student answer.\nQuestion: {question}\nStudent Answer: {student_answer}\nCorrect Answer:",
    "template_format": "f-string",
    "validate_template": true,
    "_type": "prompt"
}

In [20]:
prompt = load_prompt("prompt_with_output_parser.json")

In [21]:
prompt.output_parser.parse("George Washington was born in 1732 and died in 1799.\nScore: 1/2")

{'answer': 'George Washington was born in 1732 and died in 1799.',
 'score': '1/2'}